In [18]:
import nltk
import requests
import warnings
from sqlalchemy import text
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

# Data manipulation and analysis
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sqlalchemy.orm import sessionmaker
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta

# Financial data extraction
import yfinance as yf

# SQL Alchemy for database interaction
from sqlalchemy import create_engine

# Additional libraries for analysis or visualization
import matplotlib.pyplot as plt
import seaborn as sns
from config import api_key

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [39]:
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetches historical stock prices for the specified ticker within the given timeframe.
    """
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data


In [40]:
def fetch_all_stock_data(tickers, start_date, end_date):
    stock_data = {}
    for ticker in tickers:
        print(f"Fetching data for {ticker}")
        stock_data[ticker] = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

In [42]:
# Function to fetch headlines from NewsAPI
def fetch_news(api_key, ticker):
    base_url = "https://newsapi.org/v2/everything"
    params = {
        'q': ticker,             # Search query (ticker symbol)
        'sortBy': 'publishedAt', # Sort by publication date
        'apiKey': api_key        # Your NewsAPI key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    headlines = [article['title'] for article in articles]
    return headlines

In [43]:
def get_avg_sentiment_scores():
    textual_data = {}
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

    for ticker in tickers:
        headlines = fetch_news(api_key, ticker)
        textual_data[ticker] = headlines

    # Analyzing sentiment for each headline and averaging the scores
    average_sentiment_scores = {}

    for ticker, headlines in textual_data.items():
        sentiments = [TextBlob(headline).sentiment.polarity if headline else 0 for headline in headlines]
        average_sentiment_scores[ticker] = np.mean(sentiments)

    # Extract tickers and sentiment scores
    tickers = list(average_sentiment_scores.keys())
    sentiment_scores = list(average_sentiment_scores.values())

    pd.DataFrame(sentiment_scores, tickers,['Average Sentiment Score']).\
        to_json('static/Resources/Average_Sentiment_Score.json') 

In [44]:
def calculate_daily_return(stock_data):
    """
    Calculates the daily return from the closing prices.
    """
    stock_data['Daily_Return'] = stock_data['Close'].pct_change()
    return stock_data


In [45]:
def load_data_to_db(data, ticker):
    """
    Loads transformed data into the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)


In [46]:
def read_data(ticker):
    """
    Reads the data for the specified ticker from the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    query = f"SELECT * FROM '{ticker}'"
    data = pd.read_sql_query(query, con=engine)
    return data


In [47]:
def plot_moving_averages(stock_data, ticker):
    data = stock_data[ticker]
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    
    plt.figure(figsize=(14, 7))
    plt.plot(data['Close'], label='Close Price', alpha=0.5)
    plt.plot(data['SMA_50'], label='50-Day SMA')
    plt.plot(data['SMA_200'], label='200-Day SMA')
    plt.title(f"{ticker} - Moving Averages")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.show()

In [48]:
def predict_monthly_prices(model, last_known_price, last_known_date, months=36):
    future_prices = []
    future_dates = []
    current_price = last_known_price
    
    for month in range(months):
        # Approximating each month by 21 trading days
        for day in range(21):
            next_input = pd.DataFrame(data=[[current_price]], columns=['Previous Close'])
            current_price = model.predict(next_input)[0]
        
        future_prices.append(current_price)
        # Assuming last_known_date is a datetime object; add roughly 30 days for each month
        last_known_date += timedelta(days=30)
        future_dates.append(last_known_date)
    
    return future_dates, future_prices

In [49]:
def train_model_for_ticker(ticker_data):
    """
    Trains a LinearRegression model for the given ticker data.
    """
    data = ticker_data[['Close']].copy()
    data['Previous Close'] = data['Close'].shift(1)
    data.dropna(inplace=True)  # Drop the first row which now contains NaN

    X = data[['Previous Close']]  # Features
    y = data['Close']  # Target

    model = LinearRegression()
    model.fit(X, y)  # Training the model
    return model

In [50]:
# Objective Function (Negative Sharpe Ratio)
def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.01):
    p_var = np.dot(weights.T, np.dot(cov_matrix, weights))
    p_ret = np.dot(weights, expected_returns)
    return -(p_ret - risk_free_rate) / np.sqrt(p_var)

In [51]:
def analyze_sentiment(text):
    """Analyze the sentiment of a text and return polarity and subjectivity."""
    analysis = TextBlob(text)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity

In [52]:
def main():
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

    for ticker in tickers:
        print(f"Fetching news for: {ticker}")
        articles = fetch_news(api_key, ticker)
        if articles['status'] == 'ok':
            for article in articles['articles'][:5]:  # Show only the first 5 articles for brevity
                description = article.get('description') or article.get('title')
                polarity, subjectivity = analyze_sentiment(description)
                print(f"Ticker: {ticker}, Title: {article['title']}")
                print(f"Sentiment Polarity: {polarity:.2f}, Sentiment Subjectivity: {subjectivity:.2f}")
                print(f"URL: {article['url']}\n")
    
    return articles


In [53]:
def fetch_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="5y")
    return data['Close']

def calculate_parameters(tickers):
    prices = {ticker: fetch_data(ticker) for ticker in tickers}
    prices = pd.DataFrame(prices)
    daily_returns = prices.pct_change()
    annual_mean_returns = daily_returns.mean() * 252
    annual_std_devs = daily_returns.std() * np.sqrt(252)
    return {ticker: {'mean': annual_mean_returns[ticker], 'std': annual_std_devs[ticker]}
            for ticker in tickers}


def calculate_expected_returns(tickers):
    prices = pd.DataFrame({ticker: fetch_data(ticker) for ticker in tickers})
    daily_returns = prices.pct_change()
    average_daily_returns = daily_returns.mean()
    annual_expected_returns = average_daily_returns * 252  # Convert to annual returns
    return annual_expected_returns


In [54]:
import pandas as pd
from sqlalchemy import create_engine, text, Table, Column, Integer, String, MetaData, Float
from sqlalchemy.orm import sessionmaker

# Create engine and session factory
engine = create_engine('sqlite:///stock_market_analysis.sqlite')
Session = sessionmaker(bind=engine)

# Function to check table existence
def table_exists(engine, table_name):
    with engine.connect() as connection:
        query = text("SELECT name FROM sqlite_master WHERE type='table' AND name=:table_name")
        result = connection.execute(query, {"table_name": table_name}).fetchone()
        return result is not None

# Function to create and populate a table with sample data
def create_and_populate_table(engine, table_name):
    metadata = MetaData()
    table = Table(
        table_name, metadata,
        Column('id', Integer, primary_key=True),
        Column('date', String),
        Column('open', Float),
        Column('high', Float),
        Column('low', Float),
        Column('close', Float),
        Column('volume', Integer)
    )
    metadata.create_all(engine)
    
    # Sample data
    sample_data = [
        {'date': '2024-06-01', 'open': 100.0, 'high': 105.0, 'low': 95.0, 'close': 102.0, 'volume': 1000000},
        {'date': '2024-06-02', 'open': 102.0, 'high': 106.0, 'low': 96.0, 'close': 103.0, 'volume': 1500000},
        # Add more sample rows as needed
    ]
    
    with engine.connect() as connection:
        for row in sample_data:
            query = table.insert().values(row)
            connection.execute(query)

# List of tickers
tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

# Dictionary to store DataFrames for each ticker
dataframes = {}

# Process each ticker
for ticker in tickers:
    if not table_exists(engine, ticker):
        print(f"Table {ticker} does not exist in the database. Creating and populating the table.")
        create_and_populate_table(engine, ticker)
    else:
        with Session() as session:
            query = text(f"SELECT * FROM {ticker}")
            result = session.execute(query)
            dataframes[ticker] = pd.DataFrame(result.fetchall(), columns=result.keys())

# Display the dataframe for 'AAPL' or process further as needed
if 'AAPL' in dataframes:
    print(dataframes['AAPL'].head())
else:
    print("No data for AAPL.")


Empty DataFrame
Columns: [id, date, open, high, low, close, volume]
Index: []


In [34]:
import pandas as pd
from sqlalchemy import create_engine, text, Table, Column, Integer, String, MetaData, Float
from sqlalchemy.orm import sessionmaker

# Create engine and session factory
engine = create_engine('sqlite:///stock_market_analysis.sqlite')
Session = sessionmaker(bind=engine)

# Function to check table existence
def table_exists(engine, table_name):
    with engine.connect() as connection:
        query = text("SELECT name FROM sqlite_master WHERE type='table' AND name=:table_name")
        result = connection.execute(query, {"table_name": table_name}).fetchone()
        return result is not None

# Function to create and populate a table with sample data
def create_and_populate_table(engine, table_name):
    metadata = MetaData()
    table = Table(
        table_name, metadata,
        Column('id', Integer, primary_key=True),
        Column('date', String),
        Column('open', Float),
        Column('high', Float),
        Column('low', Float),
        Column('close', Float),
        Column('volume', Integer)
    )
    metadata.create_all(engine)
    
    # Sample data
    sample_data = [
        {'date': '2024-06-01', 'open': 100.0, 'high': 105.0, 'low': 95.0, 'close': 102.0, 'volume': 1000000},
        {'date': '2024-06-02', 'open': 102.0, 'high': 106.0, 'low': 96.0, 'close': 103.0, 'volume': 1500000},
        # Add more sample rows as needed
    ]
    
    with engine.connect() as connection:
        for row in sample_data:
            query = table.insert().values(row)
            connection.execute(query)

# List of tickers
tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

# Dictionary to store DataFrames for each ticker
dataframes = {}

# Process each ticker
for ticker in tickers:
    if not table_exists(engine, ticker):
        print(f"Table {ticker} does not exist in the database. Creating and populating the table.")
        create_and_populate_table(engine, ticker)
    else:
        with Session() as session:
            query = text(f"SELECT * FROM {ticker}")
            result = session.execute(query)
            dataframes[ticker] = pd.DataFrame(result.fetchall(), columns=result.keys())

# Display the dataframe for 'AAPL' or process further as needed
if 'AAPL' in dataframes:
    print(dataframes['AAPL'].head())
else:
    print("No data for AAPL.")


Empty DataFrame
Columns: [id, date, open, high, low, close, volume]
Index: []
